# Geotagged Only

In [ ]:
################ GEOTAGGED ONLY ##########################

# get distances column from all datasets
# first determine if there are any duplicate tweet ids in there

import pandas as pd
import tqdm

for i in tqdm.tqdm(range(0, 112)):
    df = pd.read_feather("s3://mt5599/final/processed_tweets_" + str(i) + ".feather")

    # keep tweets with no distances and NER mention
    gdf = df[pd.notnull(df.place_full_name)].reset_index()
    gdf.to_feather("s3://mt5599/dissertation/geotagged_" + str(i) + ".feather" )

In [ ]:
# combining into one dataset

gdfs = []

for i in tqdm.tqdm(range(0, 112)):
    gdf = pd.read_feather("s3://mt5599/dissertation/geotagged_" + str(i) + ".feather")

    gdfs.append(gdf)

gdf = pd.concat(gdfs, axis=0, ignore_index=True).reset_index(drop=True).reset_index()
gdf.to_feather("s3://mt5599/final/geotagged.feather")

In [ ]:
gdf.drop(["level_0", "index"], axis=1, inplace=True)

In [2]:
import pandas as pd

df = pd.read_feather("s3://mt5599/final/geotagged.feather")
df = df.drop_duplicates("id")
df

,level_0,index,id,DateTime,coordinates,place,username,user_id,user_location,tweet_content,...,place_country,place_country_code,coordinates_longitude,coordinates_latitude,ner_type,ner_word,gmaps_lat,gmaps_long,gmaps_address,distance
0,0,7,676710697422725120,2015-12-15 10:29:37+00:00,"Coordinates(longitude=-57.5592828, latitude=-3...","Place(fullName='Mar del Plata, Argentina', nam...",darcena2553,2358639522,No soy unica soy diferente,A rendir,...,Argentina,AR,-57.559283,-38.058940,None,None,NaN,NaN,None,NaN
1,1,8,674648800020033536,2015-12-09 17:56:22+00:00,"Coordinates(longitude=-57.5548276, latitude=-3...","Place(fullName='Mar del Plata, Argentina', nam...",darcena2553,2358639522,No soy unica soy diferente,¿Que paso?,...,Argentina,AR,-57.554828,-38.059752,None,None,NaN,NaN,None,NaN
2,2,9,671493815635746816,2015-12-01 00:59:36+00:00,"Coordinates(longitude=-57.5548158, latitude=-3...","Place(fullName='Mar del Plata, Argentina', nam...",darcena2553,2358639522,No soy unica soy diferente,Tu cuerpo me hace babear inundando el colegio ...,...,Argentina,AR,-57.554816,-38.059739,None,None,NaN,NaN,None,NaN
3,3,10,669967909456879616,2015-11-26 19:56:11+00:00,"Coordinates(longitude=-57.5591638, latitude=-3...","Place(fullName='Mar del Plata, Argentina', nam...",darcena2553,2358639522,No soy unica soy diferente,"Vieja del orto ,ojala que se muera y se PUDRA ...",...,Argentina,AR,-57.559164,-38.058979,None,None,NaN,NaN,None,NaN
4,4,11,668096786893021184,2015-11-21 16:01:01+00:00,"Coordinates(longitude=-57.5548388, latitude=-3...","Place(fullName='Mar del Plata, Argentina', nam...",darcena2553,2358639522,No soy unica soy diferente,"Me miras yo te miro ,me sonrreis y me matas ,m...",...,Argentina,AR,-57.554839,-38.059755,None,None,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61395063,61395063,1361566,725087645500624896,2016-04-26 22:22:20+00:00,"Coordinates(longitude=-56.7153342, latitude=-3...","Place(fullName='San Clemente del Tuyú, Argenti...",ludmi526,3224699505,"La Costa, Argentina",Un tal Martes💫 @ bellasartessanclemente https:...,...,Argentina,AR,-56.715334,-36.378340,PER,Martes,NaN,NaN,None,NaN
61395064,61395064,1361590,724364680278491136,2016-04-24 22:29:32+00:00,"Coordinates(longitude=-56.68988962, latitude=-...","Place(fullName='La Costa, Argentina', name='La...",ludmi526,3224699505,"La Costa, Argentina",Una de anoche. @ Santa Teresita La Costa https...,...,Argentina,AR,-56.689890,-36.538927,LOC,Santa Teresita La Costa,-36.544051,-56.690393,"Santa Teresita, Buenos Aires Province, Argentina",0.570435
61395065,61395065,1361591,724358546779983872,2016-04-24 22:05:10+00:00,"Coordinates(longitude=-56.68988962, latitude=-...","Place(fullName='La Costa, Argentina', name='La...",ludmi526,3224699505,"La Costa, Argentina",I need you. @ Santa Teresita La Costa https://...,...,Argentina,AR,-56.689890,-36.538927,LOC,Santa Teresita La Costa,-36.544051,-56.690393,"Santa Teresita, Buenos Aires Province, Argentina",0.570435
61395066,61395066,1361631,723655990579724288,2016-04-22 23:33:27+00:00,"Coordinates(longitude=-56.7153342, latitude=-3...","Place(fullName='San Clemente del Tuyú, Argenti...",ludmi526,3224699505,"La Costa, Argentina",!!!! @ bellasartessanclemente https://t.co/x2E...,...,Argentina,AR,-56.715334,-36.378340,None,None,NaN,NaN,None,NaN


In [3]:
df = df[0:1000]

In [ ]:
!pip install epiweeks

In [ ]:
import tqdm
from multiprocessing import Pool # multithreading
from epiweeks import Week

# granularity
def granularity_helper(row):
    
    s = row[1]["gmaps_address"]
    
    week = Week.fromdate(row[1]["DateTime"], system="iso")
    row[1]["epi_week"] = week.cdcformat()
    return row[1]



def granularity(df):
    
    pool = Pool(processes=round(len(df.index)/100))

    result_arr = []
    
    for result in tqdm.tqdm(pool.imap_unordered(granularity_helper, df.iterrows()),
                            total=len(df.index)):
        result_arr.append(result)
                
    df = pd.concat(result_arr, axis=1).transpose().sort_index()
                
    return df

In [ ]:
# add epi weeks into df

df = granularity(df)

In [ ]:
# Investigating top 20 most geotagged countries and their frequencies

loc_vc = gdf.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
loc_total = sum(loc_vc.counts)
#gdf_only = gdf[gdf.ner_type != "LOC"]
#only_loc_vc = gdf_only.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
#only_loc_total = sum(loc_only_vc.counts)

vc = pd.DataFrame(data = {"loc": [],
                          "loc_counts": [],
                          "loc_prop": [],
                    #     "only_loc": [],
                   #      "only_loc_counts": [],
                   #      "only_loc_prop": []})

n = 20
for i in range(n):
    vc.loc[i, "loc"] = loc_vc.loc[i, "unique_values"]
    vc.loc[i, "loc_counts"] = loc_vc.loc[i, "counts"]
    vc.loc[i, "loc_prop"] = loc_vc.loc[i, "counts"] / loc_total
    
  #  vc.loc[i, "only_loc"] = only_loc_vc.loc[i, "unique_values"]
  #  vc.loc[i, "only_loc_counts"] = only_loc_vc.loc[i, "counts"]
  #  vc.loc[i, "only_loc_prop"] = only_loc_vc.loc[i, "counts"] / only_loc_total
    
vc.loc[n+1, "loc"] = "Other"
vc.loc[n+1, "loc_counts"] = loc_total - sum(loc_vc.loc[0:n, "counts"])
vc.loc[n+1, "loc_prop"] = vc.loc[n+1, "loc_counts"] / loc_total
#vc.loc[n+1, "only_loc"] = "Other"
#vc.loc[n+1, "only_loc_counts"] = only_loc_total - sum(only_loc_vc.loc[0:n, "counts"])
#vc.loc[n+1, "only_loc_prop"] = vc.loc[n+1, "only_loc_counts"] / only_loc_total

vc.reset_index(inplace=True, drop=True)
#vc.drop(0, axis=0, inplace=True)
print("Table \ref{tab:geotagged-locations} shows the number of geotags by top 20 countries,",
      "with 'Other' denoting other countries (does not include un-geotagged tweets).")
vc

In [ ]:
print("For each user, the countries they geotagged over the course of the relevant time period were found.")

import itertools

user_df = df.groupby(['username'])["place_country"].unique().reset_index(name='countries')

# number of countries per user
countries_per_user = user_df.countries.len()

print()
print("The mean number of geotagged countries was ", np.mean(countries_per_user),
     "and the median was ", np.median(countries_per_user))

user_countries = user_df.countries.tolist()
all_countries = pd.DataFrame({"country":list(itertools.chain.from_iterable(user_countries))})
temp = all_countries.country.value_counts()

print("Table \ref{tab:geotagged-locations-users} shows their breakdown.")
temp

In [ ]:
print("Table \ref{tab:geotagged-locations-argentina-1} shows the top 20 geotags within Argentina that were geotagged.",
      "There were a total of \textcolor{red}{number} geotagged tweets from Argentina.",
     "Table \ref{tab:geotagged-locations-argentina-2} shows the geotags that were only geotagged (no mentions)."

top_20 = gdf.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
#top_20 = top_20.loc[,0:20]
top_20 = top_20.loc[0:20, "unique_values"].tolist()
top_20

import plotly.express as px

# with Argentina
# ^ like above code, but for people:
loc_p = gdf[gdf.place_country.isin(top_20)]
loc_p = loc_p.groupby(['place_country', "epi_week"])["id"].count().reset_index(name='counts')
#only_loc_p = gdf_loc[gdf_loc.place_country.isin(top_20)]
#only_loc_p = only_loc_p.groupby(['place_country', "epi_week"])["id"].count().reset_index(name='counts')
# plot pie chart

fig = px.line(loc_p, x="epi_week", y="counts", color="country",
                 labels={
                     "epi_week": "Epi Week",
                     "counts": "Number of Tweets",
                     "species": "Species of Iris"})
plt.savefig('geotagged_country_epi_week.png')
fig.show()

# Stalking Users

In [ ]:
# creating dictionary to see how many people stay or leave 
import tqdm

#usernames = df.username.unique()

stay_leave = pd.DataFrame(df.username.unique(), columns=["username"])
stay_leave

In [ ]:
# How many tweets per user
# How many of each user's tweets from Argentina?
from multiprocessing import Pool # multithreading

stay_leave["total_tweets"] = None
stay_leave["only_argentina"] = None

def func1(username):
    user_df = df[df.username == username]
    stay_leave.loc[stay_leave.username == username, "total_tweets"] = [user_df.shape[0]] # number of tweets over the year
    stay_leave.loc[stay_leave.username == username, "only_argentina"] = all(user_df.place_country_code == "AR") # number of people who never left the country
    return stay_leave[stay_leave.username==username]

pool = Pool(processes=100)
no_tweets = 0
dfs = []
for result in tqdm.tqdm(pool.imap_unordered(func1, stay_leave.username.unique()),
                        total=len(stay_leave.username.unique())):
    dfs.append(result)

In [ ]:
# Averages, min, max of above

import numpy as np

# number of tweets over the year
#print("mean", np.mean(stay_leave.total_tweets))
#print("mediam", np.median(stay_leave.total_tweets))
#print("max", max(stay_leave.total_tweets))
#print("min", min(stay_leave.total_tweets))
#print("mean", np.std(stay_leave.total_tweets))

stay_leave.total_tweets.describe()

In [ ]:
# number of people who never left the country, or did

stay_leave["only_argentina"] = stay_leave["total_tweets"] == stay_leave["only_argentina"]

stay_leave.only_argentina.value_counts(dropna=False)

In [ ]:
import numpy as np
import tqdm

dates = pd.date_range(start='2015-10-01', end='2016-11-01', freq='MS')

dates_arr = []
for i in range(len(dates)-1):
    dates_arr.append(dates[i:i+2]) 


for username in tqdm.tqdm(range(len(stay_leave))):
    tweeted_all_months = []
    average_tweets_per_month = []
    for i in tqdm.tqdm(range(len(dates_arr))):
        date = dates_arr[i]
        start_date = date[0]
        end_date = date[1]
        mask = (df.username == username) & (df.DateTime > start_date) & (df.DateTime <= end_date)
        tweeted_all_months[i] = tweets_this_month > 0
        average_tweets_per_month[i] = df[mask].shape[0]
        
    # people who tweeted at least once a month
    stay_leave.loc[stay_leave.username == username, "tweeted_all_months"] = all(tweeted_all_months == True)
    
    # number of average tweets per month per user
    stay_leave.loc[stay_leave.username == username, "average_tweets_per_month"] = np.mean(average_tweets_per_month)
    

In [ ]:
# number of people who tweeted at least once a month
stay_leave.tweeted_all_months.value_counts(dropna=False)

In [ ]:
# number of average tweets per month per user
stay_leave.average_tweets_per_month.value_counts(dropna=False)

## EPI Weeks

In [ ]:
from datetime import date
from epiweeks import Week

my_date = date(2018, 12, 30)
Week.fromdate(mdf_loc, system="iso")